# Projeto de Sinais e Sistemas

### Alunos: 
Aline Fortaleza (affs2)
Beatriz Leão (bgcl)
Danilo Carvalho (dlc3)
Luisa Leiria (lfla)
Rafael Azevedo (raas)

In [1]:
import os
import numpy as np
import librosa
import datasets
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset
from collections import Counter
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from concurrent.futures import ProcessPoolExecutor
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from scipy.signal import get_window
from skimage.feature import hog
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy.signal
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from hmmlearn import hmm
import speech_recognition as sr



Carregar o dataset Google Speech Commands

In [2]:
dataset = load_dataset("google/speech_commands", "v0.02", split='train', trust_remote_code=True)

Função para carregar áudio de um arquivo e aplicar normalização e trim (remover silêncio)

In [3]:
def load_audio(file_path, sample_rate=16000):
    """Carrega um arquivo de áudio e aplica normalização e trim."""
    audio, sr = librosa.load(file_path, sr=sample_rate)
    audio, _ = librosa.effects.trim(audio)
    return audio, sr

Nessa etapa primeiro o audio é pre-processado (com normalização, pre-enfase e trim), logo em seguida é aplicada a função para extrair características do audio e por fim o dataset é processado, limitando a 2900 instancias por label

In [4]:
def pre_process_audio(audio, sr, top_db=20):
    """Aplica normalização, pré-ênfase e trim ao áudio."""
    audio = librosa.util.normalize(audio)
    audio = librosa.effects.preemphasis(audio, coef=0.97)
    audio, _ = librosa.effects.trim(audio, top_db=top_db)
    return audio

def extract_features(audio, sr, n_fft=512, hop_length=256, n_mfcc=13):
    """Extrai características do Zero Crossing Rate, Transformada de Fourier, MFCC, Chroma e Spectral Contrast do áudio."""
    
    # Zero Crossing Rate
    zero_crossings = librosa.feature.zero_crossing_rate(y=audio, hop_length=hop_length)
    zcr_mean = np.mean(zero_crossings)
    zcr_std = np.std(zero_crossings)
    
    # Short-Time Fourier Transform (STFT)
    stft = np.abs(librosa.stft(audio, n_fft=n_fft, hop_length=hop_length))
    
    # STFT statistics
    fft_mean = np.mean(stft, axis=1)
    fft_var = np.var(stft, axis=1)
    fft_median = np.median(stft, axis=1)
    fft_std = np.std(stft, axis=1)
    fft_min = np.min(stft, axis=1)
    fft_max = np.max(stft, axis=1)
    
    # Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc_mean = np.mean(mfccs, axis=1)
    mfcc_std = np.std(mfccs, axis=1)
    mfcc_min = np.min(mfccs, axis=1)
    mfcc_max = np.max(mfccs, axis=1)
    
    # Chroma Feature
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length)
    chroma_mean = np.mean(chroma, axis=1)
    chroma_std = np.std(chroma, axis=1)
    chroma_min = np.min(chroma, axis=1)
    chroma_max = np.max(chroma, axis=1)
    
    # Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length)
    contrast_mean = np.mean(spectral_contrast, axis=1)
    contrast_std = np.std(spectral_contrast, axis=1)
    contrast_min = np.min(spectral_contrast, axis=1)
    contrast_max = np.max(spectral_contrast, axis=1)
    
    # Combinando todas as caracteristicas em um único vetor de características
    combined_features = np.concatenate((
        [zcr_mean, zcr_std], 
        fft_mean, fft_var, fft_median, fft_std, fft_min, fft_max,
        mfcc_mean, mfcc_std, mfcc_min, mfcc_max,
        chroma_mean, chroma_std, chroma_min, chroma_max,
        contrast_mean, contrast_std, contrast_min, contrast_max
    ))
    
    return combined_features

def process_dataset(dataset, sample_size=1000, label_limit=2900):
    """Processa um subconjunto do dataset, extraindo características, limitando a 1250 instâncias por rótulo."""
    X, y = [], []
    label_counts = defaultdict(int)
    processed_samples = 0

    subset = dataset.select(range(min(sample_size, len(dataset))))
    
    for sample in subset:
        label = sample['label']
        
        if label < 0 or label >= 10: # pegando as palavras de indice 0 a 9
            continue

        if label_counts[label] >= label_limit:
            continue
        
        audio_array = sample['audio']['array']
        sr = sample['audio']['sampling_rate']
        audio = pre_process_audio(audio_array, sr)
        features = extract_features(audio, sr)
        X.append(features)
        y.append(label)
        label_counts[label] += 1
        processed_samples += 1
        
        if processed_samples >= sample_size:
            break
    
    return np.array(X), np.array(y)

Função de treinamento

In [5]:
# Carrega o dataset Google Speech Commands com splits de treino e teste
train_dataset = load_dataset("google/speech_commands", "v0.02", split='train', trust_remote_code=True)
test_dataset = load_dataset("google/speech_commands", "v0.02", split='test', trust_remote_code=True)

# Processa uma amostra dos datasets de treino e teste
X_train, y_train = process_dataset(train_dataset, sample_size=84848)  # Ampliar a amostra
X_test, y_test = process_dataset(test_dataset, sample_size=4890)

# Verifica distribuição de classes
train_class_counts = Counter(y_train)
test_class_counts = Counter(y_test)
print(f"Distribuição de classes em y_train: {train_class_counts}")
print(f"Distribuição de classes em y_test: {test_class_counts}")

# Verifica se há mais de uma classe no conjunto de treinamento
unique_classes = len(set(y_train))
if unique_classes < 2:
    raise ValueError(f"O conjunto de treinamento deve conter pelo menos duas classes, mas tem {unique_classes}.")

# Normaliza os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Cria e treinar o modelo de Regressão Logística
modelo_regressao_logistica = LogisticRegression(max_iter=1000)
print("\nTreinando e avaliando o modelo Regressão Logística com dados não processados")
modelo_regressao_logistica.fit(X_train_scaled, y_train)
y_pred = modelo_regressao_logistica.predict(X_test_scaled)

# Avalia o modelo e imprime os resultados
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Regressão Logística: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred))


/Users/luisaleiria/anaconda3/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Distribuição de classes em y_train: Counter({3: 2900, 9: 2900, 4: 2900, 1: 2900, 7: 2900, 6: 2900, 5: 2900, 8: 2900, 2: 2900, 0: 2900})
Distribuição de classes em y_test: Counter({2: 425, 0: 419, 4: 412, 8: 411, 3: 406, 1: 405, 9: 402, 7: 402, 6: 396, 5: 396})

Treinando e avaliando o modelo Regressão Logística com dados não processados
Acurácia do modelo Regressão Logística: 76.22%
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       419
           1       0.66      0.73      0.69       405
           2       0.77      0.78      0.78       425
           3       0.73      0.69      0.71       406
           4       0.72      0.72      0.72       412
           5       0.83      0.75      0.79       396
           6       0.76      0.78      0.77       396
           7       0.76      0.77      0.77       402
           8       0.83      0.82      0.82       411
           9       0.71      0.68      0.69       402

    accuracy      

Função para capturar áudio do microfone, processar e fazer uma previsão

In [33]:
def listen_and_predict(modelo_regressao_logistica, label_names, scaler):
    # Cria um reconhecedor que reconhece a fala
    reconhecedor = sr.Recognizer()

    # Usa o microfone como fonte de áudio
    with sr.Microphone() as source:
        print("Ajustando o ruído do ambiente")
        reconhecedor.adjust_for_ambient_noise(source)

        print("Já pode falar") # podem ser identificadas as palavras : yes, no, up, down, left, right, on, off, stop go
        audio = reconhecedor.listen(source)

    # Transforma o áudio capturado em um array numpy
    audio_data = np.frombuffer(audio.get_raw_data(), dtype=np.int16).astype(np.float32) / 32768.0

    # Processa o áudio e extrair características
    sample_rate = 16000  # Taxa de amostragem
    audio_processed = pre_process_audio(audio_data, sample_rate)
    features = extract_features(audio_processed, sample_rate)

    # Normaliza as características
    features_scaled = scaler.transform(features.reshape(1, -1))

    # Prediz a palavra falada
    predicted_label_index = modelo_regressao_logistica.predict(features_scaled)[0]
    predicted_word = label_names[predicted_label_index]
    print(f"A palavra reconhecida foi: {predicted_word}")

# Fala qual foi a palavra dita com base nos labels do dataset
label_names = train_dataset.features['label'].names

listen_and_predict(modelo_regressao_logistica, label_names, scaler)

Ajustando o ruído do ambiente
Já pode falar
A palavra reconhecida foi: go
